### Update the weights to reduce loss

#### w1 = w1 - *learning rate* * d/dw1

learning rate is about 0.1

In [35]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline

df = pd.read_csv(f"{os.path.dirname(os.path.abspath('__file__'))}\\4_insurance_data.csv")
print(df.head())
df.shape

   age  affordibility  bought_insurance
0   22              1                 0
1   25              0                 0
2   47              1                 1
3   52              0                 0
4   46              1                 1


(28, 3)

In [36]:
# split train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

# apply scaling
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

X_test_scaled

,age,affordibility
2,0.47,1
10,0.18,1
21,0.26,0
11,0.28,1
14,0.49,1
9,0.61,1


In [37]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

1
Epoch 4803/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4804/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4805/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4669 - accuracy: 0.9091
Epoch 4806/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4807/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4808/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4809/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4810/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4668 - accuracy: 0.9091
Epoch 4811/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4667 - accuracy: 0.9091
Epoch 4812/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.4667 - accur

In [38]:
model.evaluate(X_test_scaled, y_test) # perfect model

1/1 [==============================] - 0s 63ms/step - loss: 0.3550 - accuracy: 1.0000


[0.3549775183200836, 1.0]

In [39]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.0608673],
        [1.4086505]], dtype=float32),
 array([-2.9137032], dtype=float32))

In [53]:

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# manual prediction funcion using sigmoid
def prediction_function(age, affordibility):
    return sigmoid(coef[0] * age + coef[1] * affordibility + intercept)

def log_loss(y_actual, y_predicted):
    eplison = 1e-15
    y_predicted_new = np.array([min(max(i, eplison), 1 - eplison) for i in y_predicted])

    return -np.mean(y_actual * np.log(y_predicted_new) + (1 - y_actual) * np.log(1 - y_predicted_new))

In [44]:
print(prediction_function(0.47, 1))
print(model.predict(X_test_scaled))

[0.70548487]


array([[0.70548487],
       [0.35569552],
       [0.16827844],
       [0.47801176],
       [0.7260696 ],
       [0.82949835]], dtype=float32)

In [54]:
def gradient_descent(age, affordibility, y_actual, epochs, loss_threshold):
    w1 = w2 = 1
    bias, rate, n = 0, 0.5, len(age)

    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordibility + bias

        # predict and calculate loss
        y_predicted = sigmoid(weighted_sum)
        loss = log_loss(y_actual, y_predicted)

        w1d = (1/n) * np.dot(np.transpose(age), (y_predicted - y_actual))
        w2d = (1/n) * np.dot(np.transpose(affordibility), (y_predicted - y_actual))

        bias_d = np.mean(y_predicted - y_actual)

        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d
        print(f"Epoch: {i}/{epochs} - w1: {w1} - w2: {w2} - bias: {bias} - loss: {loss})")

        if loss <= loss_threshold:
            break

    return w1, w2, bias

gradient_descent(X_train_scaled["age"], X_train_scaled["affordibility"], y_train, 1000, 0.4631)

1.4047108446120309 - bias: -2.2621692830941247 - loss: 0.5001933919340792)
Epoch: 202/1000 - w1: 3.4656079374887283 - w2: 1.4051944805878065 - bias: -2.2671884898023182 - loss: 0.4998869137205552)
Epoch: 203/1000 - w1: 3.4768709612509157 - w2: 1.4056728830805474 - bias: -2.2721947079716065 - loss: 0.499581686073539)
Epoch: 204/1000 - w1: 3.4881123852216054 - w2: 1.4061461626631908 - bias: -2.2771880355996448 - loss: 0.4992777027855931)
Epoch: 205/1000 - w1: 3.4993322397861726 - w2: 1.406614427814455 - bias: -2.282168569236039 - loss: 0.4989749576960279)
Epoch: 206/1000 - w1: 3.510530555686274 - w2: 1.407077784957646 - bias: -2.287136404007907 - loss: 0.4986734446900829)
Epoch: 207/1000 - w1: 3.5217073640113545 - w2: 1.407536338498725 - bias: -2.2920916336449575 - loss: 0.49837315769813556)
Epoch: 208/1000 - w1: 3.5328626961903016 - w2: 1.4079901908636536 - bias: -2.2970343505041027 - loss: 0.49807409069493364)
Epoch: 209/1000 - w1: 3.5439965839832466 - w2: 1.4084394425350295 - bias: -2

(5.051047623653049, 1.4569794548473887, -2.9596534546250037)

In [55]:
coef, intercept

(array([[5.0608673],
        [1.4086505]], dtype=float32),
 array([-2.9137032], dtype=float32))